In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import pandas as pd

In [2]:
data1=pd.read_csv('Preprocessed_thyroid_data.csv')

In [4]:
X = data1.iloc[: , 0:-1]
Y = data1.iloc[: , -1]

In [5]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state=1)

In [6]:
# Encoding categorical data
## Encoding the Independent Variable

ordinal_encoder = OrdinalEncoder()
X_train_cat_encoded = pd.DataFrame(ordinal_encoder.fit_transform(X_train.select_dtypes(exclude='number')))
X_train_cat_encoded.columns = X_train.select_dtypes(exclude='number').columns

X_test_cat_encoded = pd.DataFrame(ordinal_encoder.transform(X_test.select_dtypes(exclude='number')))
X_test_cat_encoded.columns = X_test.select_dtypes(exclude='number').columns

## Encoding the Independent Variable
label_encoder = LabelEncoder()
Y_train_cat_encoded= pd.DataFrame(label_encoder.fit_transform(Y_train))
Y_test_cat_encoded = pd.DataFrame(label_encoder.transform(Y_test))

In [7]:
## Standardization

sc = StandardScaler()
X_train_sc=pd.DataFrame(sc.fit_transform(X_train.select_dtypes(exclude='O')))
X_test_sc=pd.DataFrame(sc.transform(X_test.select_dtypes(exclude='O')))

X_train_sc.columns=X_train.select_dtypes(exclude='O').columns
X_test_sc.columns=X_test.select_dtypes(exclude='O').columns

#Combining data
X_train_final=pd.concat([X_train_sc,X_train_cat_encoded],axis=1)
X_test_final=pd.concat([X_test_sc,X_test_cat_encoded],axis=1)

In [8]:
# Handling imbalanced Dataset
#Since the dataset is small, will use over-sampling: SMOTE technique to balance the data

from imblearn.over_sampling import SMOTE

X_train_resample,Y_train_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_train_final,Y_train_cat_encoded)
X_test_resample,Y_test_resample=SMOTE(random_state=0,k_neighbors=1).fit_resample(X_test_final,Y_test_cat_encoded)

X_train_resample.shape,X_test_resample.shape,Y_train_resample.shape,Y_test_resample.shape

((7852, 28), (1476, 28), (7852, 1), (1476, 1))

In [9]:
## Feature Selection
print('Training dataset shape:', X_train_resample.shape, Y_train_resample.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample.shape)

Y_train_resample_flat = Y_train_resample.to_numpy().ravel()
Y_test_resample_flat = Y_test_resample.to_numpy().ravel()

print('Training dataset shape:', X_train_resample.shape, Y_train_resample_flat.shape)
print('Testing dataset shape:', X_test_resample.shape, Y_test_resample_flat.shape)

Training dataset shape: (7852, 28) (7852, 1)
Testing dataset shape: (1476, 28) (1476, 1)
Training dataset shape: (7852, 28) (7852,)
Testing dataset shape: (1476, 28) (1476,)


In [10]:
#Forward selection approach
rf = RandomForestClassifier(n_estimators=100, max_depth=5)
forward_fs = sfs(rf , k_features=10,forward=True,floating=False,verbose=2,scoring='accuracy',cv=5)
forward_fs = forward_fs.fit(X_train_resample, Y_train_resample_flat)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  28 out of  28 | elapsed:   30.3s finished

[2021-10-26 14:34:20] Features: 1/10 -- score: 0.7810739234614651[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:   41.6s finished

[2021-10-26 14:35:01] Features: 2/10 -- score: 0.9672691741638861[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  26 out of  26 | elapsed:   39.0s finished

[2021-10-26 14:35:40] Features: 3/10 -- score: 0.9842079571209055[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  

In [11]:
feat_names = list(forward_fs.k_feature_names_)

X_train_new=X_train_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured']]
X_test_new=X_test_resample[['age','sex','TSH', 'TT4', 'FTI', 'on_thyroxine', 'on_antithyroid_medication', 'goitre', 'hypopituitary', 'psych', 'T3_measured']]

In [12]:
#Fitting the Random Forest model
rf_model=rf.fit(X_train_new,Y_train_resample_flat)

#Checking the metrics of Random Forest
def print_Score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        pred=clf.predict(x_train)
        clf_report=pd.DataFrame(classification_report(y_train,pred,output_dict=True))
        print("Train Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_train,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("-----------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_train,pred)}\n")
    elif train==False:
        pred=clf.predict(x_test)
        clf_report=pd.DataFrame(classification_report(y_test,pred,output_dict=True))
        print("Test Result:\n===============")
        print(f"Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------")
        print(f"Classification Report:\n{clf_report}")
        print("---------------------------------")
        print(f"Confusion Matrix:\n{confusion_matrix(y_test,pred)}\n")

print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(rf_model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Train Result:
Accuracy Score:99.83%
---------------------------------
Classification Report:
                     0            1            2            3  accuracy  \
precision     0.995436     0.999488     0.998474     1.000000  0.998344   
recall        1.000000     0.993887     1.000000     0.999491  0.998344   
f1-score      0.997713     0.996679     0.999236     0.999745  0.998344   
support    1963.000000  1963.000000  1963.000000  1963.000000  0.998344   

             macro avg  weighted avg  
precision     0.998349      0.998349  
recall        0.998344      0.998344  
f1-score      0.998343      0.998343  
support    7852.000000   7852.000000  
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   9 1951    3    0]
 [   0    0 1963    0]
 [   0    1    0 1962]]

Test Result:
Accuracy Score:97.90%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.940727   

In [13]:
##Hyper parameter tuning
RF=RandomForestClassifier()
model=RF.fit(X_train_new,Y_train_resample_flat)

print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=True)
print_Score(model,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Train Result:
Accuracy Score:100.00%
---------------------------------
Classification Report:
                0       1       2       3  accuracy  macro avg  weighted avg
precision     1.0     1.0     1.0     1.0       1.0        1.0           1.0
recall        1.0     1.0     1.0     1.0       1.0        1.0           1.0
f1-score      1.0     1.0     1.0     1.0       1.0        1.0           1.0
support    1963.0  1963.0  1963.0  1963.0       1.0     7852.0        7852.0
-----------------------------------
Confusion Matrix:
[[1963    0    0    0]
 [   0 1963    0    0]
 [   0    0 1963    0]
 [   0    0    0 1963]]

Test Result:
Accuracy Score:98.10%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.946154    1.000000    1.000000   0.98103     0.982051   
recall       1.000000    0.995935    0.947154   0.98103     0.981030   
f1-score     0.972332    0.997963    0.972860   0.98103     0.981

In [14]:
## Randomized Search CV

#No of trees in Random Forest
import numpy as np
n_estimators=[int(x) for x in np.linspace(start=200,stop=2000,num=10)]
#No of features consider at every split
max_features=['auto','sqrt','log2']
#maximum no of levels in trees
max_depth=[int(x) for x in np.linspace(10,1000,10)]
#minimum no of samples required to split a node
min_samples_split=[1,3,4,5,7,9]
#minimum samples leafs required at each leaf node
min_sample_leafs=[1,2,4,6,8]

#create random gird
random_grid={'n_estimators':n_estimators,
'max_features':max_features,
'max_depth':max_depth,
'min_samples_split':min_samples_split,
'min_samples_leaf':min_sample_leafs,
'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [1, 3, 4, 5, 7, 9], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [15]:
rcv=RandomizedSearchCV(estimator=RF,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,random_state=0,n_jobs=-1)

rcv.fit(X_train_new,Y_train_resample_flat)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\Sagar\anaconda3\python\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [0.99783501 0.99490579        nan 0.99605194 0.99579715 0.99796233
 0.99617922 0.9971981  0.99579715 0.99732552 0.99541513 0.99490584
        nan 0.99566978        nan 0.9955425  0.99808971 0.99796233
        nan 0.99477842 0.99808971 0.99796238 0.99783501 0.99796238
 0.99783501        nan        nan 0.99643396 0.99566978        nan
 0.99554246        nan        nan 0.99796238 0.99528781 0.99796233
 0.99796233 0.99592448 0.99745289 0.99796233 0.99503311 0.99783501
 0.99490584 0.99745289 0.99579715 0.99745294 0.99796238 0.99541513
        nan 0.99796238 0.99783501 0.99528781 0.99745289 0.99554246
 0.99808971 0.99643396 0.99668866 0.99783501 0.99783501 0.99605194
 0.99783501 0.99796233 0.99770769        nan 0.99732557 0.99477842
 0.99452372        nan 0.99796233 0.99592452 0.99796238 0.99783501
 0.99732557 0.99668871 0.99808971 0.99707083 0

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [1, 3, 4, 5, 7, 9],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=0, verbose=2)

In [16]:
rcv.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=780, max_features='log2',
                       min_samples_split=4, n_estimators=400)

In [17]:
best_random_grid=rcv.best_estimator_
print_Score(best_random_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [18]:
param_grid = {
    'criterion': [rcv.best_params_['criterion']],
    'max_depth': [rcv.best_params_['max_depth']],
    'max_features': [rcv.best_params_['max_features']],
    'min_samples_leaf': [rcv.best_params_['min_samples_leaf'], 
                         rcv.best_params_['min_samples_leaf']+2, 
                         rcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rcv.best_params_['min_samples_split'] - 2,
                          rcv.best_params_['min_samples_split'] - 1,
                          rcv.best_params_['min_samples_split'], 
                          rcv.best_params_['min_samples_split'] +1,
                          rcv.best_params_['min_samples_split'] + 2],
    'n_estimators': [rcv.best_params_['n_estimators'] - 200, rcv.best_params_['n_estimators'] - 100, 
                     rcv.best_params_['n_estimators'], 
                     rcv.best_params_['n_estimators'] + 100, rcv.best_params_['n_estimators'] + 200]
}

In [19]:
print(param_grid)

{'criterion': ['entropy'], 'max_depth': [780], 'max_features': ['log2'], 'min_samples_leaf': [1, 3, 5], 'min_samples_split': [2, 3, 4, 5, 6], 'n_estimators': [200, 300, 400, 500, 600]}


In [20]:
grid_search=GridSearchCV(estimator=RF,param_grid=param_grid,cv=10,n_jobs=1,verbose=2)
grid_search.fit(X_train_new,Y_train_resample_flat)

Fitting 10 folds for each of 75 candidates, totalling 750 fits
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.3s
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.3s
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.3s
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.3s
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.3s
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.4s
[CV] END criterion=entropy, max_depth=780, max_features=log2, min_samples_leaf=1, min_samples_spl

GridSearchCV(cv=10, estimator=RandomForestClassifier(), n_jobs=1,
             param_grid={'criterion': ['entropy'], 'max_depth': [780],
                         'max_features': ['log2'],
                         'min_samples_leaf': [1, 3, 5],
                         'min_samples_split': [2, 3, 4, 5, 6],
                         'n_estimators': [200, 300, 400, 500, 600]},
             verbose=2)

In [21]:

best_grid=grid_search.best_estimator_
best_grid

RandomForestClassifier(criterion='entropy', max_depth=780, max_features='log2',
                       min_samples_split=4, n_estimators=600)

In [22]:
print_Score(best_grid,X_train_new,X_test_new,Y_train_resample_flat,Y_test_resample_flat,train=False)

Test Result:
Accuracy Score:97.97%
---------------------------------
Classification Report:
                    0           1           2  accuracy    macro avg  \
precision    0.942529    1.000000    1.000000  0.979675     0.980843   
recall       1.000000    0.995935    0.943089  0.979675     0.979675   
f1-score     0.970414    0.997963    0.970711  0.979675     0.979696   
support    492.000000  492.000000  492.000000  0.979675  1476.000000   

           weighted avg  
precision      0.980843  
recall         0.979675  
f1-score       0.979696  
support     1476.000000  
---------------------------------
Confusion Matrix:
[[492   0   0]
 [  2 490   0]
 [ 28   0 464]]



In [23]:
#Now, will convert our final model into pickle file

import pickle

pickle_out=open('Thyroid_model.pkl','wb')
pickle.dump(grid_search,pickle_out)
pickle_out.close()